In [1]:
import os
import pvlib
import pandas as pd

In [2]:
YEAR = 1990
STARTDATE = '%d-01-01T00:00:00' % YEAR
ENDDATE = '%d-12-31T23:59:59' % YEAR
TIMES = pd.date_range(start=STARTDATE, end=ENDDATE, freq='H')
INVERTERS = pvlib.pvsystem.retrieve_sam('CECInverter')
INVERTER_10K = INVERTERS['SMA_America__SB10000TL_US__240V_']
CECMODS = pvlib.pvsystem.retrieve_sam('CECMod')
CECMOD_POLY = CECMODS['Canadian_Solar_Inc__CS6X_300P']
CECMOD_MONO = CECMODS['Canadian_Solar_Inc__CS6X_300M']
LATITUDE, LONGITUDE = 40.5137, -108.5449
NREL_API_KEY = os.getenv('NREL_API_KEY', 'DEMO_KEY')
EMAIL = os.getenv('EMAIL', 'bwana.marko@yahoo.com')

In [3]:
header, data = pvlib.iotools.get_psm3(LATITUDE, LONGITUDE, NREL_API_KEY, EMAIL)

In [8]:
header

{'Source': 'NSRDB',
 'Location ID': '155474',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Latitude': 40.53,
 'Longitude': -108.54,
 'Time Zone': -7,
 'Elevation': 2133,
 'Local Time Zone': -7,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.1.1'}

In [6]:
data

,Year,Month,Day,Hour,Minute,Temperature,Dew Point,DHI,DNI,GHI,Surface Albedo,Pressure,Wind Direction,Wind Speed
2011-01-01 00:30:00-07:00,2011,1,1,0,30,-23.0,-25.0,0.0,0.0,0.0,0.87,780.0,252.0,1.4
2011-01-01 01:30:00-07:00,2011,1,1,1,30,-23.0,-25.0,0.0,0.0,0.0,0.87,780.0,255.0,1.4
2011-01-01 02:30:00-07:00,2011,1,1,2,30,-23.0,-26.0,0.0,0.0,0.0,0.87,780.0,254.0,1.3
2011-01-01 03:30:00-07:00,2011,1,1,3,30,-23.0,-26.0,0.0,0.0,0.0,0.87,780.0,251.0,1.3
2011-01-01 04:30:00-07:00,2011,1,1,4,30,-23.0,-26.0,0.0,0.0,0.0,0.87,780.0,249.0,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-12-31 19:30:00-07:00,2005,12,31,19,30,-19.0,-23.0,0.0,0.0,0.0,0.87,780.0,197.0,0.9
2005-12-31 20:30:00-07:00,2005,12,31,20,30,-20.0,-24.0,0.0,0.0,0.0,0.87,780.0,227.0,1.2
2005-12-31 21:30:00-07:00,2005,12,31,21,30,-21.0,-25.0,0.0,0.0,0.0,0.87,780.0,232.0,1.3
2005-12-31 22:30:00-07:00,2005,12,31,22,30,-22.0,-25.0,0.0,0.0,0.0,0.87,780.0,236.0,1.4


In [11]:
# get solar position
data.index = TIMES
sp = pvlib.solarposition.get_solarposition(
        TIMES, LATITUDE, LONGITUDE)

In [12]:
solar_zenith = sp.apparent_zenith.values
solar_azimuth = sp.azimuth.values

In [13]:
dni = data.DNI.values
ghi = data.GHI.values
dhi = data.DHI.values

In [14]:
surface_albedo = data['Surface Albedo'].values
temp_air = data.Temperature.values
dni_extra = pvlib.irradiance.get_extra_radiation(TIMES).values

In [15]:
temp_air

array([-23., -23., -23., ..., -21., -22., -22.])

In [16]:
tracker = pvlib.tracking.singleaxis(solar_zenith, solar_azimuth)

In [17]:
surface_tilt = tracker['surface_tilt']

In [18]:
surface_azimuth = tracker['surface_azimuth']

In [19]:
poa_sky_diffuse = pvlib.irradiance.get_sky_diffuse(
        surface_tilt, surface_azimuth, solar_zenith, solar_azimuth,
        dni, ghi, dhi, dni_extra=dni_extra, model='haydavies')

In [20]:
aoi = tracker['aoi']

In [21]:
poa_ground_diffuse = pvlib.irradiance.get_ground_diffuse(
        surface_tilt, ghi, albedo=surface_albedo)

In [22]:
poa = pvlib.irradiance.poa_components(
        aoi, dni, poa_sky_diffuse, poa_ground_diffuse)

In [23]:
poa_direct = poa['poa_direct']

In [24]:
poa_diffuse = poa['poa_diffuse']

In [25]:
poa_global = poa['poa_global']

In [26]:
iam = pvlib.iam.ashrae(aoi)

In [27]:
effective_irradiance = poa_direct*iam + poa_diffuse

In [28]:
temp_cell = pvlib.temperature.pvsyst_cell(poa_global, temp_air)

In [29]:
# this is the magic
cecparams = pvlib.pvsystem.calcparams_cec(
        effective_irradiance, temp_cell,
        CECMOD_MONO.alpha_sc, CECMOD_MONO.a_ref,
        CECMOD_MONO.I_L_ref, CECMOD_MONO.I_o_ref,
        CECMOD_MONO.R_sh_ref, CECMOD_MONO.R_s, CECMOD_MONO.Adjust)
mpp = pvlib.pvsystem.max_power_point(*cecparams, method='newton')
mpp = pd.DataFrame(mpp, index=TIMES)
Edaily = mpp.p_mp.resample('D').sum()